# Plotting Ballistic Ejecta from the South Pole-Aitken Impact

## Workflow:
This script runs through all the tracers in the data file and selects the tracers that follow a ballistic trajectory. It outputs the results to a csv file which can be used for plotting.

In [7]:
#import python packages 
import tools
import numpy as np
import pandas as pd

In [8]:
#import the data file. Change the path to your file :) 
data = tools.load_ballistic_data("/data2/tmd02/SPA/v15/tracer_export_all.npz", end = 398)

In [9]:
#get data
velocity_data = tools.get_velocity(data)
radial_distance_data = tools.get_radial_distance(data)
radial_velocity_data = tools.get_radial_velocity(data)
radial_acceleration_data = tools.get_radial_acceleration(data, radial_velocity_data)
average_time = tools.get_average_time(data)

In [10]:
h_launch = 150
tracer_idxs, time_steps = tools.BallisticCheckVec(data, h_launch)
# print(time_steps)
# print(tracer_idxs[398])
# print(time_steps.max())
# print(np.min(time_steps))
# print(np.max(time_steps))
# print(time_steps.shape)
ejection_times = data['time'][time_steps]

In [11]:
tracer_material = data['trm'][tracer_idxs]

In [12]:
R0 = 1750
g0 = 0.00162

g = g0 * R0**2 / (R0 + h_launch) **2
v_tilda = tools.get_v_tilda_vec(velocity_data, tracer_idxs, time_steps, R0 + h_launch, g)

In [13]:
v_r, longitude, latitude = tools.get_launch_vector_vec(data,velocity_data, tracer_idxs, time_steps)
longitude_magnitude, longitude_direction = longitude
latitude_magnitude, latitude_direction = latitude

In [14]:
theta_launch, height, distance, azimuth = tools.get_ballistic_vectors_vec(data, velocity_data, tracer_idxs, time_steps, h_launch, R0, g0)

/home/nb3018/SPA/tools/ballistics.py:105: RuntimeWarning: invalid value encountered in arccos
  beta = np.arccos(data['zmark'] / (data['r']+ 1e-12))[:, :-1]  # angle at the centre of the moon


In [15]:
print('height min =',height.min())
print('height max =',height.max())
print('height mean =',height.mean())
print('height median', np.median(height))

height min = -8975370.111943852
height max = 3620717148.1054935
height mean = 51802.61227510793
height median 116.49570269998722


In [16]:
#multiply the azimuth with -1 to get the right set up
final_latitude, final_longitude = tools.track_vec(latitude[0], longitude[0], -1 * azimuth, distance)

In [17]:
c1 = final_longitude < 0
c2 = final_longitude > 0
final_longitude[c1] = final_longitude[c1] + 180
final_longitude[c2] = final_longitude[c2] - 180
final_latitude = -1 * final_latitude 

In [18]:
def make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth):
    initial_df = pd.DataFrame({'Tracer Index' : tracer_idxs, 'Ejection Time' : ejection_times, 'Resultant Velocity' : v_r, 'Launch Longitude' : longitude[0], 'Launch Latitude' : latitude[0], 'Launch Angle' : theta_launch, 'Height' : height, 'Distance' : distance, 'Landing Longitude' : final_longitude, 'Landing Latitude' : final_latitude, 'Tracer Material' : tracer_material, 'Azimuth' : azimuth})
    return initial_df

In [19]:
df = make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth)
# print(df['Launch Longitude'][df['Launch Longitude'] < 150])
# print(df['Ejection Time'][df['Ejection Time'] > 0].min())
# print(df.Distance)
# print(df.Distance.describe())
# print(df[df.Distance < 0])
# # print(df['Ejection Time'].describe())
# print(df.loc[1360])
# print(df['Resultant Velocity'].describe())
# # print(df['Tracer Material'].value_counts())
# print(df['Resultant Velocity'].describe())

In [20]:
df = df[df['Resultant Velocity']< 2.38] 
df

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth
2,2,226.037994,1.617119,178.744432,62.602130,33.246242,1046.042924,3411.569347,-2.804542,49.087380,4.0,178.908287
14,14,204.003006,1.868617,177.284390,63.737029,54.024108,3355.144725,3008.344093,-7.763010,34.675759,4.0,175.804733
15,15,168.032806,2.188735,177.385621,65.573109,42.266348,20081.121491,3964.991817,-4.101284,64.237975,4.0,179.158699
24,26,198.023499,0.877993,175.209731,64.166427,54.310613,229.092856,691.652606,-5.961820,-41.531563,4.0,177.721726
30,524311,378.017914,2.068138,169.646332,58.505746,32.486707,7340.102145,3944.476173,-51.047004,67.092924,2.0,160.899304
...,...,...,...,...,...,...,...,...,...,...,...,...
71450,524252,558.043823,0.075180,163.355480,56.295703,25.040778,0.369124,324.239092,-23.062471,-46.452983,2.0,155.289391
71451,524280,270.074585,1.465679,175.217321,61.089890,53.567101,1023.367926,1951.645461,-10.976904,2.627826,2.0,173.106325
71452,524281,382.032593,0.565175,169.630078,58.337012,43.850013,61.423857,403.634621,-17.064453,-45.762897,2.0,159.160794
71453,524282,536.045776,0.220910,163.183415,56.646921,29.367394,4.341333,297.350388,-20.877028,-47.234341,2.0,163.481333


In [27]:
df[df['Tracer Material'] == 4.0]

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth
2,2,226.037994,1.617119,178.744432,62.602130,33.246242,1046.042924,3411.569347,-2.804542,49.087380,4.0,178.908287
14,14,204.003006,1.868617,177.284390,63.737029,54.024108,3355.144725,3008.344093,-7.763010,34.675759,4.0,175.804733
15,15,168.032806,2.188735,177.385621,65.573109,42.266348,20081.121491,3964.991817,-4.101284,64.237975,4.0,179.158699
24,26,198.023499,0.877993,175.209731,64.166427,54.310613,229.092856,691.652606,-5.961820,-41.531563,4.0,177.721726
36,38,190.016495,1.617411,173.606704,64.729308,46.513456,1384.600875,2762.214656,-8.974563,25.684464,4.0,177.673857
...,...,...,...,...,...,...,...,...,...,...,...,...
3505,5263,588.008301,0.521661,171.482425,53.907895,31.073101,28.975343,438.251246,-10.673989,-39.633823,4.0,173.284844
3510,5285,592.002319,0.512593,170.164488,53.997070,35.654139,35.416122,402.840680,-14.017608,-41.107376,4.0,166.064692
3525,5409,612.003296,0.090090,179.288763,52.438184,39.717395,1.208710,186.380596,-0.711237,-46.336001,4.0,180.000000
3540,5487,616.065796,0.517121,174.103966,53.280814,27.556064,22.905298,460.861314,-7.649067,-38.240460,4.0,174.704067


In [22]:
#save the dataframe as a csv file 
df.to_csv("full_tracer_v15.csv", index=False)    

In [23]:
mirror = df.copy()
mirror.loc[:, 'Landing Longitude'] = -1 * mirror.loc[:, 'Landing Longitude']

In [24]:
frames = [df, mirror]
result = pd.concat(frames, ignore_index=True)
result

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth
0,2,226.037994,1.617119,178.744432,62.602130,33.246242,1046.042924,3411.569347,-2.804542,49.087380,4.0,178.908287
1,14,204.003006,1.868617,177.284390,63.737029,54.024108,3355.144725,3008.344093,-7.763010,34.675759,4.0,175.804733
2,15,168.032806,2.188735,177.385621,65.573109,42.266348,20081.121491,3964.991817,-4.101284,64.237975,4.0,179.158699
3,26,198.023499,0.877993,175.209731,64.166427,54.310613,229.092856,691.652606,-5.961820,-41.531563,4.0,177.721726
4,524311,378.017914,2.068138,169.646332,58.505746,32.486707,7340.102145,3944.476173,-51.047004,67.092924,2.0,160.899304
...,...,...,...,...,...,...,...,...,...,...,...,...
100459,524252,558.043823,0.075180,163.355480,56.295703,25.040778,0.369124,324.239092,23.062471,-46.452983,2.0,155.289391
100460,524280,270.074585,1.465679,175.217321,61.089890,53.567101,1023.367926,1951.645461,10.976904,2.627826,2.0,173.106325
100461,524281,382.032593,0.565175,169.630078,58.337012,43.850013,61.423857,403.634621,17.064453,-45.762897,2.0,159.160794
100462,524282,536.045776,0.220910,163.183415,56.646921,29.367394,4.341333,297.350388,20.877028,-47.234341,2.0,163.481333


In [25]:
result.to_csv("full_tracer_mirror_v15.csv", index=False)